<a href="https://colab.research.google.com/github/rishindrasai/ExplainableAI_Assignment/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error

# data
x = np.array([1, 2, 3, 1, 2], dtype=float)
y = np.array([120, 160, 200, 130, 170], dtype=float)

# fit linear regression y = a + b*x using ordinary least squares
X = np.vstack([np.ones_like(x), x]).T
coef, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
a = float(coef[0])
b = float(coef[1])

# predictions
y_hat = a + b * x

# baseline (mean of actual y)
baseline = float(y.mean())

# SHAP value per instruction (y_hat - baseline) assigned to feature x
shap_values = y_hat - baseline

# residuals and metrics
residuals = y - y_hat
r2 = float(r2_score(y, y_hat))
rmse = float(np.sqrt(mean_squared_error(y, y_hat)))

# prepare dataframe for the prediction table
df = pd.DataFrame({
    "x (challenges)": x.astype(int),
    "actual y (users)": y.astype(int),
    "predicted y": np.round(y_hat, 3),
    "baseline (mean y)": np.round(baseline, 3),
    "SHAP value (y_hat - baseline)": np.round(shap_values, 3),
    "residual (actual - pred)": np.round(residuals, 3),
})

print("Regression equation: y_hat = a + b * x")
print(f"a (intercept) = {a:.9f}")
print(f"b (slope)     = {b:.9f}")
print()
print("Baseline (mean of y):", baseline)
print()
print("Prediction table:")
print(df.to_string(index=False))
print()
print("Model performance metrics")
print(f"R-squared: {r2:.6f}")
print(f"RMSE:      {rmse:.6f}")
print()
# Per-row interpretation lines
print("Per-row interpretations:")
for i, row in df.iterrows():
    xi = int(row["x (challenges)"])
    actual = int(row["actual y (users)"])
    pred = float(row["predicted y"])
    shapv = float(row["SHAP value (y_hat - baseline)"])
    resid = float(row["residual (actual - pred)"])
    if resid > 0:
        pred_vs_actual = "Model underpredicted (actual > predicted)."
    elif resid < 0:
        pred_vs_actual = "Model overpredicted (actual < predicted)."
    else:
        pred_vs_actual = "Exact prediction."
    direction = "increases" if shapv > 0 else ("decreases" if shapv < 0 else "no effect")
    print(f"Row {i+1}: x={xi} -> predicted {pred:.3f}. SHAP={shapv:.3f} ({direction} relative to baseline). {pred_vs_actual}")


Regression equation: y_hat = a + b * x
a (intercept) = 87.857142857
b (slope)     = 37.857142857

Baseline (mean of y): 156.0

Prediction table:
 x (challenges)  actual y (users)  predicted y  baseline (mean y)  SHAP value (y_hat - baseline)  residual (actual - pred)
              1               120      125.714              156.0                        -30.286                    -5.714
              2               160      163.571              156.0                          7.571                    -3.571
              3               200      201.429              156.0                         45.429                    -1.429
              1               130      125.714              156.0                        -30.286                     4.286
              2               170      163.571              156.0                          7.571                     6.429

Model performance metrics
R-squared: 0.973994
RMSE:      4.629100

Per-row interpretations:
Row 1: x=1 -> predicted 

# Question 2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Dataset
data = pd.DataFrame({
    "Emails": [100, 80, 120, 90, 70],
    "TopicScore": [8, 6, 9, 5, 4],
    "Attendance": [200, 160, 230, 150, 130]
})

X = data[["Emails", "TopicScore"]]
y = data["Attendance"]

# Train regression model
model = LinearRegression()
model.fit(X, y)

intercept = model.intercept_
coef_emails, coef_topic = model.coef_

# Regression equation
print(f"Regression Equation: y = {intercept:.2f} + {coef_emails:.2f}*Emails + {coef_topic:.2f}*TopicScore")

# Baseline value (mean attendance)
baseline = y.mean()
print(f"Baseline (mean attendance): {baseline:.2f}")

# Predictions
data["Predicted"] = model.predict(X)

# SHAP decomposition
data["SHAP_Emails"] = coef_emails * (data["Emails"] - X["Emails"].mean())
data["SHAP_Topic"] = coef_topic * (data["TopicScore"] - X["TopicScore"].mean())

# Validate: Predicted = Baseline + SHAP_Emails + SHAP_Topic
data["SHAP_Sum"] = baseline + data["SHAP_Emails"] + data["SHAP_Topic"]

# Compare actual vs predicted
print("\nPrediction Table:")
print(data[["Emails", "TopicScore", "Attendance", "Predicted", "SHAP_Emails", "SHAP_Topic", "SHAP_Sum"]])


Regression Equation: y = 28.26 + 0.59*Emails + 14.30*TopicScore
Baseline (mean attendance): 174.00

Prediction Table:
   Emails  TopicScore  Attendance   Predicted  SHAP_Emails  SHAP_Topic  \
0     100           8         200  201.594203     4.714976   22.879227   
1      80           6         160  161.207729    -7.072464   -5.719807   
2     120           9         230  227.681159    16.502415   37.178744   
3      90           5         150  152.801932    -1.178744  -20.019324   
4      70           4         130  126.714976   -12.966184  -34.318841   

     SHAP_Sum  
0  201.594203  
1  161.207729  
2  227.681159  
3  152.801932  
4  126.714976  


# Question 3

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

data = load_diabetes()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="Disease_Progression")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

baseline = y_train.mean()

y_pred = model.predict(X_test)

coef = model.coef_
feature_means = X_train.mean()

shap_values = pd.DataFrame((X_test - feature_means) * coef, columns=X_test.columns)

shap_values["Baseline"] = baseline
shap_values["Prediction_from_SHAP"] = shap_values.sum(axis=1)
shap_values["Predicted"] = y_pred
shap_values["Actual"] = y_test.values

shap_values["Prediction_Comparison"] = np.where(y_pred > y_test, "Over Prediction",
                                               np.where(y_pred < y_test, "Under Prediction", "Exact"))

print("Intercept:", model.intercept_)
print("Coefficients:", dict(zip(X.columns, coef)))
print("Baseline:", baseline)
print(shap_values)


Intercept: 151.34560453985995
Coefficients: {'age': np.float64(37.904021350074984), 'sex': np.float64(-241.96436231273995), 'bmi': np.float64(542.4287585162899), 'bp': np.float64(347.70384391385636), 's1': np.float64(-931.4888458835163), 's2': np.float64(518.0622769833376), 's3': np.float64(163.41998299131035), 's4': np.float64(275.3179015786484), 's5': np.float64(736.1988589046839), 's6': np.float64(48.67065743196543)}
Baseline: 153.73654390934846
          age        sex        bmi         bp          s1         s2  \
287  1.663955  10.846180  -4.307759  -5.972983 -116.970979  65.277976   
211  3.453897  10.846180  19.077692   7.195035   22.732658  -8.212397   
72   2.352394 -12.218287  -3.138486  -4.775890  -96.464023  25.693757   
321  3.591585  10.846180  27.262599  27.150567  -51.605057  19.366772   
73   0.424765 -12.218287 -11.908030  -1.184613  -36.224840  27.964983   
..        ...        ...        ...        ...         ...        ...   
255  0.011702  10.846180 -36.462754 

# Question - 4

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import shap

data = pd.DataFrame({
    "studytime": [2, 3, 1, 4, 2, 3, 1, 4, 3, 2],
    "failures": [0, 1, 0, 2, 1, 0, 3, 0, 1, 2],
    "health": [5, 3, 4, 2, 4, 5, 3, 1, 4, 2],
    "absences": [2, 10, 4, 6, 8, 3, 12, 1, 5, 7],
    "G3": [15, 12, 14, 10, 11, 16, 8, 18, 13, 9]
})

features = data.drop(columns=["G3"])
target = data["G3"]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

baseline = y_train.mean()

y_pred = model.predict(X_test)

explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)
shap_df = pd.DataFrame(shap_values.values, columns=X_test.columns)
shap_df["Baseline"] = baseline
shap_df["Prediction_from_SHAP"] = shap_df.sum(axis=1) + baseline
shap_df["Predicted"] = y_pred
shap_df["Actual"] = y_test.values
shap_df["Prediction_Comparison"] = np.where(y_pred > y_test, "Over Prediction",
                                            np.where(y_pred < y_test, "Under Prediction", "Exact"))

print("Intercept:", model.intercept_)
print("Coefficients:", dict(zip(X_train.columns, model.coef_)))
print("Baseline:", baseline)
print(shap_df)

Intercept: 17.537911869789596
Coefficients: {'studytime': np.float64(0.043491685413083916), 'failures': np.float64(-2.555467337126729), 'health': np.float64(-0.6023113228353406), 'absences': np.float64(-0.1342243394645135)}
Baseline: 12.142857142857142
   studytime  failures    health  absences   Baseline  Prediction_from_SHAP  \
0   0.031065  0.365067 -0.602311  0.095875  12.142857             24.175410   
1   0.031065  0.365067 -0.000000 -0.575247  12.142857             24.106599   
2   0.031065  2.920534 -1.204623  0.364323  12.142857             26.397014   

   Predicted  Actual Prediction_Comparison  
0  12.032553      13      Under Prediction  
1  11.963742      12      Under Prediction  
2  14.254157      16      Under Prediction  
